In [ ]:
# Part 0 - Setup & quick sanity
from agent_core import new_conversation, chat_once, LEADS_CSV, FEEDBACK_LOG
from pathlib import Path

hist = new_conversation()
print("Conversation ready. Leads file:", LEADS_CSV)
print("Feedback file:", FEEDBACK_LOG)


In [ ]:
# Part 1 - Ask about mission (should answer from docs)
hist, reply = chat_once(hist, "What is your mission?")
print(reply)


In [ ]:
# Part 2 - Provide email; model should call record_customer_interest
hist, reply = chat_once(hist, "I'm interested. My name is Lina, email lina@example.com. Can we talk?")
print(reply)

# check file was written
print("Leads file exists:", Path(LEADS_CSV).exists())
print(Path(LEADS_CSV).read_text(encoding="utf-8"))


In [ ]:
# Part 3 - Ask a question not in docs (forces record_feedback)
hist, reply = chat_once(hist, "Do you have a Dubai office address and a price list?")
print(reply)

print("Feedback log exists:", Path(FEEDBACK_LOG).exists())
print(Path(FEEDBACK_LOG).read_text(encoding="utf-8"))
